In [2]:
# from flask_other.app import app
from flask_socketio import disconnect as dc
from flask import Flask
from flask_socketio import SocketIO, emit,join_room,leave_room
from flask_socketio import disconnect
from threading import Lock
import os,sys

app=Flask("demo")
from flask import  render_template,request
import time
import re

import pandas as pd
import numpy as np
import time
import re
import requests

import atexit

from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.triggers.interval import IntervalTrigger

logger = Logger('Websocket-Flask',level=ENV.NODE_LOG_LEVEL.value).logger



##################################
async_mode = None
app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app, async_mode=async_mode)

thread = None
thread_lock = Lock()
clients = []
sentences = []

name_space = '/chat'

@app.route('/')
def index():
    return render_template('index.html', async_mode=socketio.async_mode)


@socketio.on('client_send',namespace=name_space)
def client_msg(msg):  
    uid = request.sid
    sentence = msg.get('data')
    sentence = decode(sentence)
    sentences.append(sentence)
    response = cache.chat(uid, sentence)
    if response == 'end':
        socketio.emit('my_response',{'data':'感谢选择江苏逸能，再见！'},room = uid, namespace=name_space)
        disconnect_frontend(uid)
    elif response is None:
        socketio.emit('my_response',{'data':'当前会话已经过期，感谢选择江苏逸能，再见！'},room = uid, namespace=name_space)
        disconnect_frontend(uid)
    else:
        socketio.emit('my_response',{'data':response},room = uid, namespace=name_space)

def decode(msg):
    msg = re.sub(r'%u', r'\u', msg)
    msg = msg.encode('latin-1').decode('unicode_escape')
    return msg

#sned message to a specific user
def unique_message(uid):
    socketio.emit('unique_messgae',{'data':'are you still here'}, room=uid, namespace=name_space)
    
def disconnect_frontend(uid):
    socketio.emit('my_response',{'status':'disconnected'},room = uid, namespace=name_space)


@socketio.on('disconnect',namespace=name_space)
def disconnect():
    uid = request.sid
    cache.remove_session(uid)
    leave_room(uid)
    dc()
    logger.info('{} has been disconnected from API'.format(uid))
    
    
@socketio.on('connect',namespace=name_space)
def connect():
    
    uid = request.sid
    clients.append(uid)
    logger.info('{} is trying to connect!'.format(uid))
    
    join_room(uid)
    if cache.create_session(uid):
        logger.info('{} join connection successfully'.format(uid))
        response = cache.chat(uid, '')
        if response is not None:
            socketio.emit('my_response',{'data':response},room = uid,namespace=name_space) #the first sentence
            return None
        
    else:
        logger.info('{} cannot join connection'.format(uid))
        socketio.emit('my_response',{'data':'server busy. please click new conv'},room = uid,namespace=name_space) 
        disconnect_frontend(uid)
    



In [ ]:
print('http://10.0.24.31:8899/')
print('http://0.0.0.0:6006/')
scheduler = BackgroundScheduler()
scheduler.start()
scheduler.add_job(
    func=cache.purge_inactive,
    trigger=IntervalTrigger(seconds=3),
    id='purge_cache',
    name='purge_inactive',
    replace_existing=True)
# Shut down the scheduler when exiting the app
atexit.register(lambda: scheduler.shutdown())

socketio.run(app,'0.0.0.0',6006)

http://10.0.24.31:8899/
http://0.0.0.0:6006/


2018-07-23 04:38:47,362 - INFO - CLASS:Websocket-Flask- METHOD:connect -LINE:90 - MSG:fec24e68273044cebb885dd8f98e4dc8 is trying to connect!
2018-07-23 04:38:47,366 - DEBUG - CLASS:PF- METHOD:_load_default -LINE:344 - MSG:profile is None. The default demo profile will be loaded!
2018-07-23 04:38:47,371 - INFO - CLASS:PF- METHOD:_load_default -LINE:365 - MSG:Customer ID is 1000000000, principal is 50,000, apr is 9%
2018-07-23 04:38:47,445 - INFO - CLASS:PF- METHOD:_loadUpLowBound -LINE:399 - MSG:Load profile Upper bound successfully!
2018-07-23 04:38:47,447 - INFO - CLASS:PF- METHOD:_loadUpLowBound -LINE:406 - MSG:Load profile Lower bound successfully!


Time Zone is set from ENV: Asia/Shanghai


2018-07-23 04:38:47,984 - INFO - CLASS:Cache- METHOD:create_session -LINE:53 - MSG:New session was created: fec24e68273044cebb885dd8f98e4dc8
2018-07-23 04:38:47,986 - INFO - CLASS:Cache- METHOD:create_session -LINE:54 - MSG:Remaining session number is: 999
2018-07-23 04:38:47,988 - INFO - CLASS:Websocket-Flask- METHOD:connect -LINE:94 - MSG:fec24e68273044cebb885dd8f98e4dc8 join connection successfully
2018-07-23 04:38:47,990 - INFO - CLASS:Cache- METHOD:chat -LINE:86 - MSG:receive message from user: fec24e68273044cebb885dd8f98e4dc8 ====================
2018-07-23 04:38:47,991 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:585 - MSG:Current node name is s0
2018-07-23 04:38:47,994 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:590 - MSG:Output label is 0
2018-07-23 04:38:48,001 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:605 - MSG:Next node name is cf_s1_n1_identity_q.
2018-07-23 04:38:48,003 - INFO - CLASS:Cache- METHOD:chat -LINE:90 - MSG:processing messages for user fec24e682

InitClassifier


...

IDClassifier


2018-07-23 04:38:54,928 - INFO - CLASS:Cache- METHOD:chat -LINE:86 - MSG:receive message from user: fec24e68273044cebb885dd8f98e4dc8 ====================
2018-07-23 04:38:54,931 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:585 - MSG:Current node name is cf_s1_n5_ifAcquainted_q
2018-07-23 04:38:54,943 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:590 - MSG:Output label is 1
2018-07-23 04:38:54,949 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:605 - MSG:Next node name is cf_s1_n102_ifAcquainted_s.
2018-07-23 04:38:54,951 - INFO - CLASS:Cache- METHOD:chat -LINE:90 - MSG:processing messages for user fec24e68273044cebb885dd8f98e4dc8 has been done!----------------


IfKnowDebtor


2018-07-23 04:39:25,391 - INFO - CLASS:Cache- METHOD:chat -LINE:86 - MSG:receive message from user: fec24e68273044cebb885dd8f98e4dc8 ====================
2018-07-23 04:39:25,394 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:585 - MSG:Current node name is cf_s1_n102_ifAcquainted_s
2018-07-23 04:39:25,395 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:587 - MSG:Reach Stop Node: cf_s1_n102_ifAcquainted_s
2018-07-23 04:39:25,398 - INFO - CLASS:Cache- METHOD:chat -LINE:90 - MSG:processing messages for user fec24e68273044cebb885dd8f98e4dc8 has been done!----------------
2018-07-23 04:39:25,456 - WARNING - CLASS:DB- METHOD:insert -LINE:67 - MSG:database is not enabled!
2018-07-23 04:39:25,459 - INFO - CLASS:Cache- METHOD:remove_session -LINE:79 - MSG:fec24e68273044cebb885dd8f98e4dc8 session is inactive, it has been removed!
2018-07-23 04:39:25,508 - ERROR - CLASS:Cache- METHOD:remove_session -LINE:70 - MSG:Fail to save cache for uid: 
2018-07-23 04:39:25,560 - INFO - CLASS:Websocket-Flas

KeyboardInterrupt: 

In [20]:
import time
import gc
import sys, os
import datetime as dt

ENV_PATH = '../../../ENV/'
LOG_PATH = '../../../Lib/'
sys.path.append(ENV_PATH)
sys.path.append(LOG_PATH)
from env import ENV
from LOG import Logger
from MGODB import DB



class Cache:
    def __init__(self, model_dict,max_session=1000,debug=False,host=None,port=None,enableDB=False):
        self.max_session = 1000
        self.inform_interval = 60
        self.inactive_maxlength = 150
        #{'uid': {'strategy': Tree(), 'time_response': <time>, 'time_inform': <>}
        self.active_session = {}
        self.model_dict = model_dict
        self.debug=debug
        self.enableDB = enableDB
        self.log = Logger(self.__class__.__name__,level=ENV.NODE_LOG_LEVEL.value).logger
        self.db=DB(host,port,debug,True,enable=self.enableDB)
        self._print()
        
        
    def _print(self):
        self.log.info('Max num of session is: {}'.format(self.max_session))
        self.log.info('inform inacitve interval is {} seconds'.format(self.inform_interval))
        self.log.info('inactive max length is {} seconds'.format(self.inactive_maxlength))
        if self.debug:
            self.log.info('DEBUG is enabled')
        
        
    def create_session(self, uid, profile=None):
        if len(self.active_session) < self.max_session:
            self.active_session[uid] = {}
            try:
                self.active_session[uid].update({'strategy':TreeStage1(
                                                                       debug=self.debug,
                                                                       profile=profile)})
            except KeyError as e:
                self.log.error('Key {} does not exist in profile'.format(e))
                self.log.error('create session for user {} failed'.format(uid))
                return False
                
            self.active_session[uid].update({'time_response':time.time()})
            self.active_session[uid].update({'time_inform':time.time()})
            self.log.info('New session was created: {}'.format(uid))
            self.log.info('Remaining session number is: {}'.format(self.max_session-len(self.active_session)))
            return True
        else:
            return False
        
    def remove_session(self,uid):
        response = '您当前的会话超过 {} 秒没有响应，系统将关闭当前会话！如有需求，请开始新的对话！'.format(self.inactive_maxlength)
        try:
            socketio.emit('my_response',{'data':response},room = uid, namespace=name_space)
        except:
            pass
        
        try:
            history = self.active_session[uid]['strategy'].cache.copy()
            self.db.insert(history)
        except:
            self.log.error('Fail to save cache for uid: ')
        try:
            disconnect_frontend(uid)
        except:
            pass
       
        
        try:
            del self.active_session[uid]
            self.log.info('{} session is inactive, it has been removed!'.format(uid))
        except KeyError:
                pass
        gc.collect()
            
    def chat(self,uid,sentence):
        if self.active_session.get(uid) is not None:
            self.log.info('receive message from user: {} ===================='.format(uid))
            response = self.active_session[uid]['strategy'].process(sentence, self.model_dict)
            self.active_session[uid]['time_response'] = time.time()
            self.active_session[uid]['time_inform'] = time.time()
            self.log.info('processing messages for user {} has been done!----------------'.format(uid))
        else:
            response = None
        
        return response
        
    
    
    def _bulk_deletes(self):
        current = time.time()
        remove_list = []
        try:
            for uid in self.active_session:
                try:
                    if current - self.active_session[uid]['time_response'] > self.inactive_maxlength:
                        remove_list.append(uid)
                except KeyError:
                    pass
        except RuntimeError as e:
            self.log.error(e)
            return False
        finally:
            # delete
            for uid in remove_list:              
                self.remove_session(uid)
        return True
    
    def _bulk_inform(self):
        current = time.time()
        inform_list = []
        try:
            for uid in self.active_session:
                try:
                    if current - self.active_session[uid]['time_inform'] > self.inform_interval:
                        inform_list.append(uid)
                     
                except KeyError as e:
                        self.log.error(e)
                        pass
            for each in inform_list:
                self.inform_inactive(each)
            
        except RuntimeError:
            
            return False
        
        return True
    
    
    def purge_inactive(self):
        current = time.time()
        while True:
            if self._bulk_deletes():
                break

        while True:
            if self._bulk_inform():
                break
        
                
        
    def inform_inactive(self, uid):
        self.active_session[uid]['time_inform'] = time.time()
        response = '您有在听我说吗?请回答我刚才的问题！'
        socketio.emit('my_response',{'data':response},room = uid, namespace=name_space)
        self.log.info('{} is inactive. Just inform that user'.format(uid))
          
################################################
import sys,os
sys.path.append('../../../MLModel/code/OneClickTraining/')
sys.path.append('../../../MLModel/code/TreeModelV2/')
from all_model_py import *
import pickle
from chatbotv1 import *



models_list = ['IDClassifier','CutDebt','IfKnowDebtor','WillingToPay','Installment','ConfirmLoan']
savedModel_path = '../../../MLModel/savedModel/{}/{}.pickle'
need_set_TIMEZONE = ['CutDebt','WillingToPay','Installment','ConfirmLoan']

model_dict = {}
for each_model in models_list:
    model_dict[each_model] = pickle.load(open(savedModel_path.format(each_model,each_model), 'rb'))
    model_dict[each_model].classify('再说一次')
    if each_model in need_set_TIMEZONE:
        model_dict[each_model].re_time._set_timeZone()
    
model_dict['StopClassifier'] = StopClassifier()
model_dict['InitClassifier'] = InitClassifier()    

#################################################################
cache = Cache(
              model_dict=model_dict,debug=True,enableDB=True)

Time Zone is set from ENV: Asia/Shanghai
Time Zone is set from ENV: Asia/Shanghai
Time Zone is set from ENV: Asia/Shanghai


2018-07-24 17:58:24,981 - INFO - CLASS:DB- METHOD:_load_client -LINE:35 - MSG:get mongodb client. host is:chatbotdb, port is: 27017
2018-07-24 17:58:24,981 - INFO - CLASS:DB- METHOD:_load_client -LINE:35 - MSG:get mongodb client. host is:chatbotdb, port is: 27017
2018-07-24 17:58:24,987 - INFO - CLASS:DB- METHOD:_get_db -LINE:43 - MSG:mongodb database is: chatbotdb_debug
2018-07-24 17:58:24,987 - INFO - CLASS:DB- METHOD:_get_db -LINE:43 - MSG:mongodb database is: chatbotdb_debug
2018-07-24 17:58:24,992 - INFO - CLASS:DB- METHOD:_get_collection -LINE:51 - MSG:mongodb collection is: chat_debug
2018-07-24 17:58:24,992 - INFO - CLASS:DB- METHOD:_get_collection -LINE:51 - MSG:mongodb collection is: chat_debug
2018-07-24 17:58:24,996 - INFO - CLASS:Cache- METHOD:_print -LINE:32 - MSG:Max num of session is: 1000
2018-07-24 17:58:24,996 - INFO - CLASS:Cache- METHOD:_print -LINE:32 - MSG:Max num of session is: 1000
2018-07-24 17:58:25,002 - INFO - CLASS:Cache- METHOD:_print -LINE:33 - MSG:infor

Time Zone is set from ENV: Asia/Shanghai


In [2]:
t = TreeStage1(debug=False)

2018-07-24 17:55:13,250 - DEBUG - CLASS:PF- METHOD:_load_default -LINE:206 - MSG:profile is None. The default demo profile will be loaded!
2018-07-24 17:55:13,252 - INFO - CLASS:PF- METHOD:_load_default -LINE:227 - MSG:Customer ID is 1000000000, principal is 50,000, apr is 9%
2018-07-24 17:55:13,275 - INFO - CLASS:PF- METHOD:_loadUpLowBound -LINE:261 - MSG:Load profile Upper bound successfully!
2018-07-24 17:55:13,276 - INFO - CLASS:PF- METHOD:_loadUpLowBound -LINE:268 - MSG:Load profile Lower bound successfully!


Time Zone is set from ENV: Asia/Shanghai


In [18]:
t.process('没钱',model_dict)

2018-07-24 17:57:44,114 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:460 - MSG:Current node name is cf_s1_n105_noResult_s
2018-07-24 17:57:44,119 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:462 - MSG:Reach Stop Node: cf_s1_n105_noResult_s


'end'

In [21]:
cache.db.insert(t.cache)

InvalidDocument: Cannot encode object: 2

In [38]:
cache.db.insert({'chat':a['chat'][12]})

InvalidDocument: Cannot encode object: 3

In [22]:
t.cache

{'startTime': datetime.datetime(2018, 7, 25, 1, 55, 13, 278202, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>),
 'chat': [{'id': 1,
   'agent': '你好，这里是平安E贷小额贷款，请问是李明先生吗？',
   'customer': '是的',
   'currentNode': 'cf_s1_n1_identity_q',
   'nextNode': 'cf_s1_n2_confirmLoan_q',
   'label': 0,
   'confidence': 0.9164749818991843,
   'confidence_other': -1.0,
   'responseTime': datetime.datetime(2018, 7, 25, 1, 55, 32, 899631, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>),
   'nodeSentiment': 1},
  {'id': 2,
   'agent': '您好，我是平安E贷小额贷款的催收员小张。您之前于2018年1月16日向平安E贷小额贷款借贷了人民币50,000约定于2018年5月16日还清。截至今天已经逾期70天，逾期总欠款人民币52,000元，其中利息1,500元，滞纳金500块。现在平安E贷小额贷款委托我方协助追回欠款！请问这笔欠款为什么到现在都没有处理呢？',
   'customer': '我没借过',
   'currentNode': 'cf_s1_n2_confirmLoan_q',
   'nextNode': 'cf_s1_n2_confirmLoan_q',
   'label': 1,
   'confidence': 0.8385241377254887,
   'confidence_other': -1.0,
   'responseTime': datetime.datetime(2018, 7, 25, 1, 55, 44, 412589, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD

In [3]:
t = cache.active_session[1000]['strategy']

In [10]:
n = t.nodes['cf_s1_n1_identity_q']

In [11]:
n.output_label

109

In [9]:
t.process('',model_dict)

2018-07-20 03:11:51,444 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:534 - MSG:Current node name is cf_s1_n1_identity_q
2018-07-20 03:11:51,460 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:539 - MSG:Output label is 109
2018-07-20 03:11:51,465 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:550 - MSG:Next node name is cf_s1_n1_identity_q.


'请你不要说这些有的没的，您是李明先生吗？<-current node is: cf_s1_n1_identity_q-><-output label is: 109-><-next node is: cf_s1_n1_identity_q->'

In [4]:
c = model_dict['IDClassifier']
c.classify('天气不错')

{'label': 109, 'pred_prob': array([[0.01752875, 0.01967167, 0.96279958],
        [0.04673296, 0.05362745, 0.8996396 ],
        [0.02181207, 0.02078342, 0.9574045 ]]), 'av_pred': array([0.02869126, 0.03136085, 0.93994789])}

In [1]:
t = TreeStage1(graph_path=graph_path,msg_path=msg_path,debug=True)

NameError: name 'TreeStage1' is not defined

In [5]:

print(t.process('不行',model_dict))


2018-07-20 21:21:47,053 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:538 - MSG:Current node name is cf_s1_n1_identity_q
2018-07-20 21:21:47,064 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:543 - MSG:Output label is 1
2018-07-20 21:21:47,070 - DEBUG - CLASS:TreeStage1- METHOD:process -LINE:558 - MSG:Next node name is cf_s1_n5_ifAcquainted_q.


不好意思，打扰了，请问您认识李明先生吗！<-current node is: cf_s1_n1_identity_q-><-output label is: 1-><-next node is: cf_s1_n5_ifAcquainted_q->


In [46]:
t.cache

{'startTime': datetime.datetime(2018, 7, 25, 1, 55, 13, 278202, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>),
 'chat': [{'id': 1,
   'agent': '你好，这里是平安E贷小额贷款，请问是李明先生吗？',
   'customer': '是的',
   'currentNode': 'cf_s1_n1_identity_q',
   'nextNode': 'cf_s1_n2_confirmLoan_q',
   'label': 0,
   'confidence': 0.9164749818991843,
   'confidence_other': -1.0,
   'responseTime': datetime.datetime(2018, 7, 25, 1, 55, 32, 899631, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>),
   'nodeSentiment': 1},
  {'id': 2,
   'agent': '您好，我是平安E贷小额贷款的催收员小张。您之前于2018年1月16日向平安E贷小额贷款借贷了人民币50,000约定于2018年5月16日还清。截至今天已经逾期70天，逾期总欠款人民币52,000元，其中利息1,500元，滞纳金500块。现在平安E贷小额贷款委托我方协助追回欠款！请问这笔欠款为什么到现在都没有处理呢？',
   'customer': '我没借过',
   'currentNode': 'cf_s1_n2_confirmLoan_q',
   'nextNode': 'cf_s1_n2_confirmLoan_q',
   'label': 1,
   'confidence': 0.8385241377254887,
   'confidence_other': -1.0,
   'responseTime': datetime.datetime(2018, 7, 25, 1, 55, 44, 412589, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD

In [17]:
model = model_dict['WillingToPay']

In [4]:
cache.active_session

{'1507ae35ca504635a25246bc7630ca01': {'strategy': <chatbotv1.TreeStage1 at 0x7fab58c639b0>,
  'time_response': 1532120656.6885388,
  'time_inform': 1532120656.6885505}}

In [5]:
h = cache.active_session['1507ae35ca504635a25246bc7630ca01']['strategy'].cache.copy()

In [6]:
aa = h.copy()
aa

{'startTime': datetime.datetime(2018, 7, 20, 21, 4, 11, 535371),
 'chat': [{'id': 1,
   'agent': '不好意思，打扰了，请问您认识李明先生吗！<-current node is: cf_s1_n1_identity_q-><-output label is: 1-><-next node is: cf_s1_n5_ifAcquainted_q->',
   'customer': '不',
   'currentNode': 'cf_s1_n1_identity_q',
   'nextNode': 'cf_s1_n5_ifAcquainted_q',
   'label': 1,
   'confidence': 1,
   'confidence_other': -1,
   'responseTime': datetime.datetime(2018, 7, 20, 21, 4, 16, 687198)}],
 'status': 'incomplete'}

In [33]:
del aa['chat']

In [7]:
type(aa['chat'][0]['label'])

numpy.int64

In [14]:
type(aa['chat'][0]['confidence'])

int

In [8]:
del aa['chat'][0]['label']
# del aa['chat'][0]['nextNode']

In [9]:
aa

{'startTime': datetime.datetime(2018, 7, 20, 21, 0, 15, 620008),
 'chat': [{'id': 1,
   'agent': '不好意思，打扰了，请问您认识李明先生吗！<-current node is: cf_s1_n1_identity_q-><-output label is: 1-><-next node is: cf_s1_n5_ifAcquainted_q->',
   'customer': '不',
   'currentNode': 'cf_s1_n1_identity_q',
   'nextNode': 'cf_s1_n5_ifAcquainted_q',
   'confidence': 1,
   'confidence_other': -1,
   'responseTime': datetime.datetime(2018, 7, 20, 21, 0, 19, 721780)}],
 'status': 'incomplete'}

In [11]:
db = DB()

2018-07-20 21:01:23,471 - INFO - CLASS:DB- METHOD:_load_client -LINE:34 - MSG:get mongodb client. host is:chatbotdb, port is: 27017
2018-07-20 21:01:23,471 - INFO - CLASS:DB- METHOD:_load_client -LINE:34 - MSG:get mongodb client. host is:chatbotdb, port is: 27017
2018-07-20 21:01:23,471 - INFO - CLASS:DB- METHOD:_load_client -LINE:34 - MSG:get mongodb client. host is:chatbotdb, port is: 27017
2018-07-20 21:01:23,476 - INFO - CLASS:DB- METHOD:_get_db -LINE:42 - MSG:mongodb database is: chatbotdb
2018-07-20 21:01:23,476 - INFO - CLASS:DB- METHOD:_get_db -LINE:42 - MSG:mongodb database is: chatbotdb
2018-07-20 21:01:23,476 - INFO - CLASS:DB- METHOD:_get_db -LINE:42 - MSG:mongodb database is: chatbotdb
2018-07-20 21:01:23,482 - INFO - CLASS:DB- METHOD:_get_collection -LINE:51 - MSG:mongodb collection is: chat
2018-07-20 21:01:23,482 - INFO - CLASS:DB- METHOD:_get_collection -LINE:51 - MSG:mongodb collection is: chat
2018-07-20 21:01:23,482 - INFO - CLASS:DB- METHOD:_get_collection -LINE:51

In [42]:
dicta = a['chat'][0]
print(dicta)
for key in dicta:
    print('key: {}  --- value: {}   --- type:{}'.format(key,dicta[key],type(dicta[key])))


{'id': 1, 'agent': '你好，这里是平安E贷小额贷款，请问是李明先生吗？', 'customer': '是的', 'currentNode': 'cf_s1_n1_identity_q', 'nextNode': 'cf_s1_n2_confirmLoan_q', 'label': 0, 'confidence': 0.9164749818991843, 'confidence_other': -1.0, 'responseTime': datetime.datetime(2018, 7, 25, 1, 55, 32, 899631, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>), 'nodeSentiment': 1}
key: id  --- value: 1   --- type:<class 'int'>
key: agent  --- value: 你好，这里是平安E贷小额贷款，请问是李明先生吗？   --- type:<class 'str'>
key: customer  --- value: 是的   --- type:<class 'str'>
key: currentNode  --- value: cf_s1_n1_identity_q   --- type:<class 'str'>
key: nextNode  --- value: cf_s1_n2_confirmLoan_q   --- type:<class 'str'>
key: label  --- value: 0   --- type:<class 'int'>
key: confidence  --- value: 0.9164749818991843   --- type:<class 'float'>
key: confidence_other  --- value: -1.0   --- type:<class 'float'>
key: responseTime  --- value: 2018-07-25 01:55:32.899631+08:00   --- type:<class 'datetime.datetime'>
key: nodeSentiment  --- value: 1  

In [45]:
dicta = a['chat'][4]
print(dicta)
for key in dicta:
    print('key: {}  --- value: {}   --- type:{}'.format(key,dicta[key],type(dicta[key])))


{'id': 5, 'agent': '您之前借贷了人民币50,000块钱，约定于2018年5月16日还清，截至今天已经逾期70天，逾期总欠款人民币52,000，其中利息1,500，滞纳金500块。平安E贷小额贷款已经对您进行了多次催促，但是您依旧没有按时偿还所欠款项。现在我们正式对您下达通知，要求您在明天下午3点之前还清所有欠款。', 'customer': '没钱', 'currentNode': 'cf_s1_n15_verifyWill_q', 'nextNode': 'cf_s1_n15_verifyWill_q', 'label': 1, 'confidence': 0.9735921455854383, 'confidence_other': -1.0, 'responseTime': datetime.datetime(2018, 7, 25, 1, 57, 24, 74249, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>), 'nodeSentiment': 1}
key: id  --- value: 5   --- type:<class 'int'>
key: agent  --- value: 您之前借贷了人民币50,000块钱，约定于2018年5月16日还清，截至今天已经逾期70天，逾期总欠款人民币52,000，其中利息1,500，滞纳金500块。平安E贷小额贷款已经对您进行了多次催促，但是您依旧没有按时偿还所欠款项。现在我们正式对您下达通知，要求您在明天下午3点之前还清所有欠款。   --- type:<class 'str'>
key: customer  --- value: 没钱   --- type:<class 'str'>
key: currentNode  --- value: cf_s1_n15_verifyWill_q   --- type:<class 'str'>
key: nextNode  --- value: cf_s1_n15_verifyWill_q   --- type:<class 'str'>
key: label  --- value: 1   --- type:<class 'int'>
key: confidence  --- valu